In [ ]:
from diffusion_3d.chestct.autoencoder.vae.config import get_config
from diffusion_3d.datasets.ct_rate import CTRATEDataModule, CTRATEDataset
from diffusion_3d.utils.visualize import plot_scans

In [ ]:
cfg = get_config()
cfg

Munch({'data': Munch({'csvpath': '/raid3/arjun/ct_pretraining/csvs/sources.csv', 'datapath': '/raid3/arjun/ct_pretraining/scans/', 'checkpointspath': '/raid3/arjun/checkpoints/adaptive_autoencoder/', 'limited_dataset_size': None, 'allowed_spacings': ((0.4, 7), (-1, -1), (-1, -1)), 'allowed_shapes': ((96, -1), (256, -1), (256, -1)), 'train_augmentations': Munch({'_target_': 'monai.transforms.Compose', 'transforms': [Munch({'_target_': 'vision_architectures.transforms.croppad.CropForegroundWithCropTrackingd', 'keys': ['image'], 'source_key': 'image', 'allow_smaller': True}), Munch({'_target_': 'monai.transforms.ScaleIntensityRanged', 'keys': ['image'], 'a_min': -1000, 'a_max': 2000, 'b_min': -1.0, 'b_max': 1.0, 'clip': True}), Munch({'_target_': 'vision_architectures.transforms.croppad.RandSpatialCropSamplesWithCropTrackingd', 'keys': ['image'], 'roi_size': (76, 76, 76), 'max_roi_size': (115, 115, 115), 'random_size': True, 'num_samples': 5}), Munch({'_target_': 'vision_architectures.tra

# Train dataset and dataloader

In [ ]:
dataset = CTRATEDataset(cfg.data, "train")

train:   0%|          | 0/48996 [00:00<?, ?it/s]

No. of train datapoints: 48993


In [ ]:
plots = []
for _ in range(4):
    image = dataset[1][0]["image"]
    print(image.shape)
    plots.append(image[0])

plot_scans(plots)

torch.Size([1, 96, 96, 96])
torch.Size([1, 96, 96, 96])
torch.Size([1, 96, 96, 96])
torch.Size([1, 96, 96, 96])


interactive(children=(IntSlider(value=0, description='z', max=95), Output()), _dom_classes=('widget-interact',…

In [ ]:
dataset[1][0]


{
    'SeriesUID': 'train_9077_a_2',
    'StudyUID': 'train_9077',
    'Modality': 'ct',
    'BodyPart': 'chest',
    'Source': 'ct_rate',
    'Spacing': (0.75, 0.7188, 0.7188),
    'Shape': tensor([349, 630, 564], dtype=torch.int32),
    'FilePath': 'ct_rate/batch_0/train_9077_a_2.safetensors',
    'SliceThickness': 0.75,
    'image': metatensor([[[[-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          ...,
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000]],

         [[-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -0.7399, -0.9996, -1

In [ ]:
dataloader = CTRATEDataModule(cfg.data).train_dataloader()
for batch in dataloader:
    print(batch["image"].shape)
    break

train:   0%|          | 0/48996 [00:00<?, ?it/s]

No. of train datapoints: 48993


Calculating sample weights:   0%|          | 0/48993 [00:00<?, ?it/s]

torch.Size([20, 1, 96, 96, 96])


# Val dataset and dataloader

In [ ]:
dataset = CTRATEDataset(cfg.data, "valid")

valid:   0%|          | 0/1189 [00:00<?, ?it/s]

No. of valid datapoints: 1189


In [ ]:
for i in range(3):
    image = dataset[5 * i][0]["image"]
    print(image.shape)
    plot_scans(image[0])

torch.Size([1, 96, 96, 96])


interactive(children=(IntSlider(value=0, description='z', max=95), Output()), _dom_classes=('widget-interact',…

torch.Size([1, 96, 96, 96])


interactive(children=(IntSlider(value=0, description='z', max=95), Output()), _dom_classes=('widget-interact',…

torch.Size([1, 96, 96, 96])


interactive(children=(IntSlider(value=0, description='z', max=95), Output()), _dom_classes=('widget-interact',…

In [ ]:
dataloader = CTRATEDataModule(cfg.data).val_dataloader()
for batch in dataloader:
    print(batch["image"].shape)
    break

valid:   0%|          | 0/1189 [00:00<?, ?it/s]

No. of valid datapoints: 1189
torch.Size([20, 1, 96, 96, 96])


In [ ]:
import torch

from diffusion_3d.chestct.autoencoder.vae.nn import AdaptorResidualConnection

a = AdaptorResidualConnection(0.5)
a.set_num_steps(10)
a.weight_scheduler.step()
a.weight_scheduler.step()
a.weight_scheduler.step()
a.weight_scheduler.step()
a.weight_scheduler.step()

in1, in2 = torch.rand(2, 5, 1, 4, 4, 4)
out = a(in1, in2)
print(out.shape)

torch.Size([5, 1, 4, 4, 4])
